# Learning Signal & Ignoring Noise: Introduction to Regularization & Batching

In this Chapter, we will:
- Learn about overfitting
- Study and implement dropout
- Implement Batch Gradient Descent

> [John von Neumann] "With four Parameters I can fit an Elephant, & with five I can make him wiggle his trunk."

## 3-Layer Network on MNIST
### Let's return to the MNIST Dataset & Attempt to Classify it with the New Network

Let's start by examining the following question: "how do we know if the network is learning a good correlation?"

We saw in a previous chapter that is we froze one weight then train the network until the error is sufficiently small, then unfroze the weight in an attempt to optimize against it. In that case it won't change because the network had already learnt the existing correlation in the data.

Another important question pops-up: "What if the network had figured out a way to accurately predict the games in the training dataset, but it somehow forgot to include a valuable input?".

Neural networks always prefer shortcuts and easy-ways to 0 ERROR, and so the more powerful the neural network (more layers & weights), the more prone it is to overfit. For this reason we say that **overfitting is extremely common in neural networks**. 

We're going to study the basics of **regularization**, which is a key to combatting overfitting in neural networks. We're going to train our latest & greatest neural network with 3 layers on the MNIST dataset:

In [2]:
import sys
import numpy as np

In [3]:
from tensorflow.keras import datasets

In [4]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
X_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [5]:
X, y = X_train[:1000].reshape(1000, 28*28)/255., y_train[:1000]; X.shape

(1000, 784)

In [6]:
# we one-hot encode `y`
y_ = np.zeros((y.shape[0], 10))

In [7]:
for index, value in enumerate(y):
    y_[index][value] = 1
y = y_

In [8]:
# same for test set
X_test, y_test = X_test[:1000].reshape(1000, 28*28)/255., y_test[:1000]
one_hot_y_test = np.zeros((y_test.shape[0], 10))
for index, value in enumerate(y_test):
    one_hot_y_test[index][value] = 1

In [9]:
def softmax(v):
    bottom = np.sum([np.exp(v_i) for v_i in v])
    return np.array([np.exp(v_i)/bottom for v_i in v])

relu = lambda x: (x >= 0) * x
grad_relu = lambda x: (x >= 0)
sigmoid = lambda x: 1/(1+np.exp(-x)) 
lr, epochs, hidden_size, total_pixels, num_labels = 0.005, 300, 100, 784, 10

In [10]:
# We Initialize two weight matrices
W_1 = (0.2 * np.random.random((total_pixels,hidden_size))) - 0.1
W_2 = (0.2*np.random.random((hidden_size,num_labels))) - 0.1

In [12]:
for j in range(epochs):
    error, correct_count = (0.0, 0)
    
    for i in range(len(X)):
        layer_0 = X[i:i+1]
        layer_1 = relu(layer_0.dot(W_1))
        layer_2 = softmax(layer_1.dot(W_2))
        
        correct_count += int(np.argmax(layer_2) == np.argmax(y[i:i+1]))
        
        layer_2_delta = (layer_2 - y[i:i+1])
        layer_1_delta = layer_2_delta.dot(W_2.T)*grad_relu(layer_1)
        
        error += np.sum(layer_2_delta ** 2)
        
        W_2 -= lr * layer_1.T.dot(layer_2_delta) 
        W_1 -= lr * layer_0.T.dot(layer_1_delta)
    
    if j % 10 == 0:
        print(f"I:{j} | Error: {round(error/float(len(X)), 5)} | Correct: {round(correct_count/float(len(X)), 5)}")

I:0 | Error: 0.7074 | Correct: 0.589
I:10 | Error: 0.07741 | Correct: 0.955
I:20 | Error: 0.02303 | Correct: 0.996
I:30 | Error: 0.00596 | Correct: 1.0
I:40 | Error: 0.00216 | Correct: 1.0
I:50 | Error: 0.00102 | Correct: 1.0
I:60 | Error: 0.00057 | Correct: 1.0
I:70 | Error: 0.00035 | Correct: 1.0
I:80 | Error: 0.00024 | Correct: 1.0
I:90 | Error: 0.00017 | Correct: 1.0
I:100 | Error: 0.00012 | Correct: 1.0
I:110 | Error: 9e-05 | Correct: 1.0
I:120 | Error: 7e-05 | Correct: 1.0
I:130 | Error: 6e-05 | Correct: 1.0
I:140 | Error: 5e-05 | Correct: 1.0
I:150 | Error: 4e-05 | Correct: 1.0
I:160 | Error: 3e-05 | Correct: 1.0
I:170 | Error: 3e-05 | Correct: 1.0
I:180 | Error: 2e-05 | Correct: 1.0
I:190 | Error: 2e-05 | Correct: 1.0
I:200 | Error: 2e-05 | Correct: 1.0
I:210 | Error: 2e-05 | Correct: 1.0
I:220 | Error: 1e-05 | Correct: 1.0
I:230 | Error: 1e-05 | Correct: 1.0
I:240 | Error: 1e-05 | Correct: 1.0
I:250 | Error: 1e-05 | Correct: 1.0
I:260 | Error: 1e-05 | Correct: 1.0
I:270 | Erro

## Well, That was easy!
### The Neural Network learned to predict all 1,000 images

We've reached 100% accuracy on the sample of 1000 images, but how about measuring the performance on data that wasn't part of the original sample of 1,000 images?

Let's Evaluate the network on the test set:

In [13]:
error, correct_count = (0.0, 0)

for i in range(len(X_test)):
    layer_0 = X_test[i:i+1]
    layer_1 = relu(layer_0.dot(W_1))
    layer_2 = softmax(layer_1.dot(W_2))
    
    correct_count += int(np.argmax(layer_2) == np.argmax(one_hot_y_test[i:i+1]))
    layer_2_delta = (layer_2 - one_hot_y_test[i:i+1])
    error += np.sum(layer_2_delta ** 2)

print(f"Error: {round(error/float(len(X_test)), 5)} | Correct: {round(correct_count/float(len(X_test)), 5)}")

Error: 0.27238 | Correct: 0.827


`0.83` is called the **test accuracy**. It is important because it simulates how the neural network will do in production (the real world). This is the score that matters for measuring the actual performance of the neural network.  

Even though the accuracy is $82.7\%$, the performance is nowhere near the $100\%$. So why does the network perform bad on new testing images when it already learned to predict with a $100\%$ accuracy on the training set?

## Memorization vs. Generalizatio
### Memorizing 1,000 images is easier than generalizing to all images

Asking another question might shed some light to what's happening: "If we trained the NN to predict labels on 1,000 images, which it did perfectly, why does it work on other images at all?". The answer is that the NN is guaranteed to work well on a new image only if the new image is nearly identical to the images in the training set, Because the NN learned to transform the input data to output data for a specific data set with a specific overall configuration.

If the NN works only on nearly identical data points (in comparison to training set), then what's the. purpose of it anyway? 

What we want is a network that can work well on images different from the training set, this is what we call **Generalization**.

## Overfitting in Neural Networks
### Neural Networks can get worse if you train them too much!

For some reason, the test accuracy goes up for the first 20 iterations and then slowly decreased as the network trained more and more. This is common in neural networks. 

Overfitting is over-optimizing for the training data points, **Just like when we're molding a material for 3 forks we keep molding them until we get a very specific shape that works for all 3 but has nothing to do with the shape of a general-purpose fork**. In the same way, **we can visualize weights as high dimensional Shapes optimized to fit inputs**. As we train, weight shapes molds around the shape of the data, learning one pattern after another.

A more official definition of a neural network that overfits: A Neural Network that has **learned the noise** in the dataset.

## Where Overfitting Comes from?
### What causes neural networks to overfit?

Let's consider these two dog pictures:

<div style="text-align:center;"><img style="width:500px;" src="static/imgs/08/Dogs.png" /></div>

**Everything** that makes these pictures **unique** **beyond** what captures **the essense of "Dog" is** included in the term **"Noise"**. Examples:
- On the Left: The Pillow & the background are both Noise.
- On the Right: the Blackness can also be considered Noise (It's really the edges that tells you it's a Dog).

So how do we get NNs to train only on the signal (the essense of a dog) & Ignore the Noise?
One Way is **Early Stopping**. It turns out a large amount of noise comes in the fine grained details of an image and most of the signal is found on the general shape & perhaps color of the image. Since the neural network will exploit general patterns first, we can just stop training before it start optimizing for the fine-grained details in the image.

## The Simplest Regularization: Early Stopping
### Stop training the network when it starts getting worse (on the test data)

Early Stopping is the cheapest form of regularization, and you're in a pinch, it can be quite effective.
**Regularization** in itself is a subset of methods for getting a model to generalize to new data points instead of just memorizing the training data. In other words, regularization helps the neural network learn the signal and ignore the noise.

We often regularize a network by **increasing the difficulty** for a model to learn the fine-grained details in teh training data.

Early stopping uses a **validation set** while training, and stop when the validation score gets worse.

## Industry Standard Regularization: Dropout
### The Method: Randomly turn off neurons (setting them to 0) during training

Dropout causes the neural network to train execlusively using **random subsections of the neural network**. This regularization method is generally accepted as the go-to, state-of-the-art technique for the vast majority of networks. Its methodology is simple & inexpensive, although the intuitions behind why it works are a bit more complex. So why dropout works?
- Dropout makes a big network train like a little one by randomly training little subsections of the network at a time, and little networks don't overfit.
- Small neural networks have a smaller number of weights, meaning the network's hypothesis space is small and do not have much expressive power.

Small Neural Networks have enough room to only capture the big, obvious, high-level features. The Notion of room/capacity is very important to keep in our minds.

**Dropout** is an effective way for us to get the power of a large neural network with the resistance to overfitting of a small neural network.

## Why Dropout works? Ensembling works?
### Dropout is also a form of training a bunch of networks and averaging them

Although it's likely that large, unregularized neural networks will overfit to noise, It's unlikely they will overfit to the same noise because their weights are intiailized randomly. 

What is interesting is that neural networks, even though they're randomly generated, still start by learning the biggest, most broadly sweeping features before learning much about the noise.

The premise behind dropout is that if we allowed a bunch of overfitted neural networks to vote equally, their noise will tend to cancel out, revealing only what they all learned in common which is **The Signal**!

## Dropout in Code
### Here's how to use dropout in the real world

In [16]:
for j in range(epochs):
    error, correct_count = (0.0, 0)
    
    for i in range(len(X)):
        layer_0 = X[i:i+1]
        layer_1 = relu(layer_0.dot(W_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = layer_1.dot(W_2)
    
        correct_count += int(np.argmax(layer_2) == np.argmax(y[i:i+1]))
        
        layer_2_delta = (layer_2 - y[i:i+1])
        layer_1_delta = layer_2_delta.dot(W_2.T)*grad_relu(layer_1)
        layer_1_delta *= dropout_mask
        
        error += np.sum(layer_2_delta ** 2)
        
        W_2 -= lr * layer_1.T.dot(layer_2_delta) 
        W_1 -= lr * layer_0.T.dot(layer_1_delta)
    
    if j % 10 == 0:
        test_error, test_correct_count = 0, 0
        
        for i in range(len(X_test)):
            layer_0 = X_test[i:i+1]
            layer_1 = relu(layer_0.dot(W_1))
            layer_2 = softmax(layer_1.dot(W_2))

            test_correct_count += int(np.argmax(layer_2) == np.argmax(one_hot_y_test[i:i+1]))
            layer_2_delta = (layer_2 - one_hot_y_test[i:i+1])
            error += np.sum(layer_2_delta ** 2)
        
        print(f"I: {j} | Test-Err: {round(test_error/ float(len(X_test)), 5)} | Test-Acc: {round(test_correct_count/ float(len(X_test)), 5)} | Train-Err: {round(error/ float(len(X)), 5)} | Train-Acc: {round(correct_count/ float(len(X)), 5)}")

I: 0 | Test-Err: 0.0 | Test-Acc: 0.431 | Train-Err: 33.14016 | Train-Acc: 0.21
I: 10 | Test-Err: 0.0 | Test-Acc: 0.683 | Train-Err: 1.76192 | Train-Acc: 0.603
I: 20 | Test-Err: 0.0 | Test-Acc: 0.711 | Train-Err: 1.50841 | Train-Acc: 0.663
I: 30 | Test-Err: 0.0 | Test-Acc: 0.72 | Train-Err: 1.38042 | Train-Acc: 0.702
I: 40 | Test-Err: 0.0 | Test-Acc: 0.731 | Train-Err: 1.3698 | Train-Acc: 0.725
I: 50 | Test-Err: 0.0 | Test-Acc: 0.758 | Train-Err: 1.34658 | Train-Acc: 0.731
I: 60 | Test-Err: 0.0 | Test-Acc: 0.761 | Train-Err: 1.3256 | Train-Acc: 0.732
I: 70 | Test-Err: 0.0 | Test-Acc: 0.765 | Train-Err: 1.26724 | Train-Acc: 0.77
I: 80 | Test-Err: 0.0 | Test-Acc: 0.784 | Train-Err: 1.27603 | Train-Acc: 0.766
I: 90 | Test-Err: 0.0 | Test-Acc: 0.782 | Train-Err: 1.29988 | Train-Acc: 0.788
I: 100 | Test-Err: 0.0 | Test-Acc: 0.76 | Train-Err: 1.27171 | Train-Acc: 0.788
I: 110 | Test-Err: 0.0 | Test-Acc: 0.801 | Train-Err: 1.27866 | Train-Acc: 0.784
I: 120 | Test-Err: 0.0 | Test-Acc: 0.757 | T

**A dropout mask uses what's called a 50% bernoulli distribution.**

In evaluation mode, we multiply `layer_1` by 2, we do this because `layer_2` will perform a weighted sum of `layer_1` and even though the sum is weighted, It's still a sum over the values of `layer_1`. If we turn off half of the nodes in `layer_1`, `layer_2` would increase its sensitivity to `layer_1`. However, at test time, we no longer would need dropout, this would throw off `layer_2` sensitivity. That is why we multiply `layer_1` by 2.

After introducing Dropout, Not only the network peak at a score of 80%, it also doesn't overfit nearly as badly. We should also notice that the dropout slows down training accuracy, it previously converged to 100% pretty fastly, now, it finishes at 80%.

**Dropout is Noise**. We are introducing noise to the network to help it concentrate its training on the true signal and avoid memorizing feature-specific noise.

## Batch Gradient Descent
### Here's a Method for increasing the Speed of training & the rate of convergence 

Also called **mini-batched stochastic gradient descent**, it became a standard and largely taken for granted in neural network training.

Previously, we trained for one data example each iteration. But with Mini-batch SGD, we can train 100 training examples at a time, **averaging the weight updates among all 100 examples**. As it turns out, **individual training examples are very noisy** in terms of the weight updates they generate. Thus, averaging them makes for a smoother learning process.

Let's do this in code:

In [18]:
batch_size = 100
lr, epochs = .001, 300

In [23]:
import sys
import numpy as np

from tensorflow.keras import datasets

np.random.seed(1)

In [24]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

In [25]:
X, y = (X_train[:1000].reshape((1000, -1))/255.), y_train[:1000]
X_test = (X_test.reshape((X_test.shape[0], -1))/255.)

In [26]:
# one-hotting y
one_hots = np.zeros((y.shape[0], 10))
for i, _ in enumerate(one_hots):
    one_hots[i][y[i]] = 1
y = one_hots

In [27]:
# one-hotting y
one_hots = np.zeros((y_test.shape[0], 10))
for i, _ in enumerate(one_hots):
    one_hots[i][y_test[i]] = 1
y_test = one_hots

In [28]:
def relu(x):
    return (x >= 0) * x

def grad_relu(x):
    return x >= 0

In [29]:
batch_size = 100
lr, epochs = .001, 300
pixels_per_image, num_labels, hidden_size = 784, 10, 100

In [31]:
# Initialize Weights
W_1 = .2 * np.random.random((pixels_per_image, hidden_size)) - .1
W_2 = .2 * np.random.random((hidden_size, num_labels)) - .1

In [32]:
for j in range(epochs):
    error, correct_cnt = (0.0, 0)
    for i in range(int(len(X) / batch_size)):
        batch_start, batch_end = ((i * batch_size),((i+1)*batch_size))

        layer_0 = X[batch_start:batch_end]
        layer_1 = relu(np.dot(layer_0,W_1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = np.dot(layer_1,W_2)

        error += np.sum((y[batch_start:batch_end] - layer_2) ** 2)
        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(y[batch_start+k:batch_start+k+1]))

            layer_2_delta = (y[batch_start:batch_end]-layer_2)/batch_size
            layer_1_delta = layer_2_delta.dot(W_2.T)* grad_relu(layer_1)
            layer_1_delta *= dropout_mask

            W_2 += lr * layer_1.T.dot(layer_2_delta)
            W_1 += lr * layer_0.T.dot(layer_1_delta)
            
    if(j%10 == 0):
        test_error = 0.0
        test_correct_cnt = 0

        for i in range(len(X_test)):
            layer_0 = X_test[i:i+1]
            layer_1 = relu(np.dot(layer_0, W_1))
            layer_2 = np.dot(layer_1, W_2)

            test_error += np.sum((y_test[i:i+1] - layer_2) ** 2)
            test_correct_cnt += int(np.argmax(layer_2) == np.argmax(y_test[i:i+1]))

        sys.stdout.write("\n" + \
                         "I:" + str(j) + \
                         " Test-Err:" + str(test_error/ float(len(X_test)))[0:5] +\
                         " Test-Acc:" + str(test_correct_cnt/ float(len(X_test)))+\
                         " Train-Err:" + str(error/ float(len(X)))[0:5] +\
                         " Train-Acc:" + str(correct_cnt/ float(len(X))))


I:0 Test-Err:0.815 Test-Acc:0.3832 Train-Err:1.284 Train-Acc:0.165
I:10 Test-Err:0.568 Test-Acc:0.7173 Train-Err:0.591 Train-Acc:0.672
I:20 Test-Err:0.510 Test-Acc:0.7571 Train-Err:0.532 Train-Acc:0.729
I:30 Test-Err:0.485 Test-Acc:0.7793 Train-Err:0.498 Train-Acc:0.754
I:40 Test-Err:0.468 Test-Acc:0.7877 Train-Err:0.489 Train-Acc:0.749
I:50 Test-Err:0.458 Test-Acc:0.793 Train-Err:0.468 Train-Acc:0.775
I:60 Test-Err:0.452 Test-Acc:0.7995 Train-Err:0.452 Train-Acc:0.799
I:70 Test-Err:0.446 Test-Acc:0.803 Train-Err:0.453 Train-Acc:0.792
I:80 Test-Err:0.451 Test-Acc:0.7968 Train-Err:0.457 Train-Acc:0.786
I:90 Test-Err:0.447 Test-Acc:0.795 Train-Err:0.454 Train-Acc:0.799
I:100 Test-Err:0.448 Test-Acc:0.793 Train-Err:0.447 Train-Acc:0.796
I:110 Test-Err:0.441 Test-Acc:0.7943 Train-Err:0.426 Train-Acc:0.816
I:120 Test-Err:0.442 Test-Acc:0.7966 Train-Err:0.431 Train-Acc:0.813
I:130 Test-Err:0.441 Test-Acc:0.7906 Train-Err:0.434 Train-Acc:0.816
I:140 Test-Err:0.447 Test-Acc:0.7874 Train-Err:0

The Concept behind Mini-batch GD is the fact that we don't alter the network's parameters each data point prediction at a time. We instead predict a whole batch, get an average delta and update all parameters. This means we get rid of point level noise, by averging a batch of prediction, we have a better sense of what direction we should move the internal parameters.

Notice that the learning rate is 20 times larger than before. This is because we are much more confident in the direction the weights should take to change. In other terms, since we are taking an average of a noisy signal (the average weight change over 100 training examples), we can take bigger steps.

We'll generally see **batching ranging from 8 to as high as 2024**. Generally, Researchers take numbers randomly until they find a `batch_size` & `lr` pair that seems to work well.

In the following chapters, we'll pivot from sets of tools that universally applicable to nearly all neural networks, to special purpose architectures that are advantageous for mdeling specific types of data types.

---